# Import packages

In [ ]:
from matplotlib import pyplot as plt
from pathlib import Path
from PIL import Image
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, jaccard_score
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd


In [ ]:
sets=['B0', 'B1', 'deeplabv3p_resnet50', 'unet', 'unet_3plus', 'LDA', 'RF', 'SVC', 'FRF', 'MLP', 'DT-SegNet']
root_path = Path('Output')

### Convert pseudo_color_prediction to numpy array

In [ ]:
for current_set in sets + ['groundtruth']:
    data_dir = root_path / current_set
    output_root = data_dir
    for img_path in list(data_dir.glob('*.png')):
        print(f'Processing {str(img_path)}')
        img = Image.open(img_path)
        output = np.array(img)
        output[output>=1]=1
        data = np.array(output)
        new_data = data.astype('uint8')
        print(data.shape, new_data.shape, np.unique(new_data))
        #plt.imshow(new_data, interpolation='nearest', cmap='Greys')
        # white background
        #plt.show()
        np.save(output_root / f'{img_path.stem}.npy', new_data)
        Image.fromarray(
            new_data*255).save(output_root / f'{img_path.stem}.png')

In [ ]:
Path('Analysis').mkdir(exist_ok=True, parents=True)

## Calculate metrics

In [ ]:
sub_arr = []
file_arr = []
accuracy_score_arr = []
precision_score_arr = []
recall_score_arr = []
ssim_arr = []
f1_score_arr = []
jaccard_score_arr = []

In [ ]:
for sub in sets:
    path = Path(f'Output/{sub}')
    files = list(path.glob('*.npy'))
    for file in files:
        gt = f'Output/groundtruth/{file.stem}.npy'
        gt_data=np.load(gt)
        data=np.load(file)
        sub_arr.append(sub)
        file_arr.append(file.stem)
        accuracy_score_arr.append(
            accuracy_score(gt_data.flatten(), data.flatten())*100
        )
        precision_score_arr.append(
            precision_score(gt_data.flatten(), data.flatten())*100
        )
        recall_score_arr.append(
            recall_score(gt_data.flatten(), data.flatten())*100
        )
        f1_score_arr.append(
            f1_score(gt_data.flatten(), data.flatten())*100
        )
        jaccard_score_arr.append(
            jaccard_score(gt_data.flatten(), data.flatten())*100
        )
        ssim_arr.append(
            ssim(gt_data, data, data_range=1)*100
        )

In [ ]:
df = pd.DataFrame({
    'sub': sub_arr,
    'file': file_arr,
    'accuracy_score': accuracy_score_arr,
    'precision_score': precision_score_arr,
    'recall_score': recall_score_arr,
    'jaccard_score': jaccard_score_arr,
    'structural_similarity': ssim_arr,
    'f1_score': f1_score_arr,
    })
df


In [ ]:
# group by software
df.groupby('sub').mean(numeric_only=True)


In [ ]:
df.groupby('sub').std()

## Calculate precipitate scales distribution

In [ ]:
# precipitate scale in ROI
for cset in ['train', 'val', 'test']:
    path = Path('/root/autodl-tmp/DT_SegNet/Dataset/segmentation_labels/') / cset
    big_root = Path('/root/autodl-tmp/DT_SegNet/Dataset/segmentation_labels')
    files = list(path.glob('*.png'))

    bg_cnt_arr = []
    ppt_cnt_arr = []
    ratio_arr = []
    for file in tqdm(files):
        img = Image.open(file)
        np_data = np.asarray(img)
        big_name = file.stem.split('_')[0]
        big_path = big_root / f'{big_name}.png'
        
        bigimg = Image.open(big_path)
        bignp_data = np.asarray(bigimg)

        bigarea = bignp_data.shape[0] * bignp_data.shape[1]

        bg_cnt, ppt_cnt = np.count_nonzero(np_data == 0), np.count_nonzero(np_data == 1)

        assert np_data.size == np.count_nonzero(np_data == 0) + np.count_nonzero(np_data == 1)
        #assert bg_cnt > ppt_cnt
        ratio_arr.append(100*ppt_cnt/bigarea)

        bg_cnt_arr.append(bg_cnt)
        ppt_cnt_arr.append(ppt_cnt)

    np.savetxt(f'Analysis/{cset}_hist.txt', ratio_arr)

    ppt_ratio = np.array(ppt_cnt_arr).sum() / (np.array(bg_cnt_arr).sum()+np.array(ppt_cnt_arr).sum())

    print(cset, ppt_ratio)


## Generate visual inspection

In [ ]:
Path('/root/autodl-tmp/DT_SegNet/Analysis/Error').mkdir(exist_ok=True, parents=True)

for isuffix in suffix:
    for iid in iids:
        gt_filename = f'/root/autodl-tmp/DT_SegNet/Output/{iid}_groundtruth.npy'
        filename = f'/root/autodl-tmp/DT_SegNet/Output/{iid}_{isuffix}.npy'
        output_filename = f'/root/autodl-tmp/DT_SegNet/Analysis/Error/{iid}_{isuffix}_error'
        print(filename)
        data = np.load(filename)
        gt_data = np.load(gt_filename)

        data = data.astype('uint8')
        gt_data = gt_data.astype('uint8')
        error = np.bitwise_xor(data, gt_data)
        error_cls = np.bitwise_and(error, gt_data)
        error = error + error_cls
        plt.imshow(error, interpolation='nearest', cmap='Greys')
        # white is background
        plt.show()
        Image.fromarray(error*127).save(f'{output_filename}.png')


## Generate red error overlay

In [ ]:
error_path = Path('/root/autodl-tmp/DT_SegNet/Analysis/Error')
files = list(error_path.glob('*.png'))

In [ ]:
def add_overlay(np_data: np.ndarray, gt_data: np.ndarray, colorset: dict):
    output = np_data.copy()
    for y in range(np_data.shape[0]):
        for x in range(np_data.shape[1]):
            pt = gt_data[y,x]
            if pt in colorset:
                clr = colorset[pt]
                ca=output[y,x,0]
                cb=output[y,x,1]
                cc=output[y,x,2]
                fa=min(clr[0]+ca, 255)
                fb=min(clr[1]+cb, 255)
                fc=min(clr[2]+cc, 255)
                ma=fa-ca
                mb=fb-cb
                mc=fc-cc
                output[y,x,0]+=ma
                output[y,x,1]+=mb
                output[y,x,2]+=mc
    return output


In [ ]:
Path('/root/autodl-tmp/DT_SegNet/Analysis/Error_Overlay').mkdir(exist_ok=True, parents=True)
for filename in tqdm(files):
    data = Image.open(filename)
    print(filename)
    np_data = np.array(data)
    oid=filename.stem.split('_')[0]
    opath=f'/root/autodl-tmp/DT_SegNet/Dataset/test/{oid}.png'
    o_data = Image.open(opath)
    np_odata = np.array(o_data)
    overlay_error = add_overlay(np_odata, np_data, {
        127: (255,0,0),
        254: (0,0,255),
    })
    Image.fromarray(overlay_error, mode='RGB').save(
        f'/root/autodl-tmp/DT_SegNet/Analysis/Error_Overlay/{filename.stem}.png')
    plt.imshow(overlay_error, interpolation='nearest', cmap='Greys')
    # white is background
    plt.show()
